In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataloader import make_dataloaders
from model import ResNet
from model import ConvNet
from model import SE_ResNet
import time

In [ ]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_list = os.listdir("../Models")
print(model_list)

In [ ]:
#Define hyperparameters and model
batch_size = 256 
num_epochs = 10
model_choice = "ConvNet" #"SE_ResNet" "ResNet" "ConvNet"
n_features = 64
height = 128
width = 256
droprate = 0.6
lr = 0.01
num_blocks = 0
r = 16
    
train_loader,test_loader = make_dataloaders(height, width, batch_size)

#initialize model and sent to device
if model_choice == "SE_ResNet":
    model = SE_ResNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate, num_blocks=num_blocks, r=r).float()
    model.to(device)
    print("SE_ResNet initialized")
    
elif model_choice == "ResNet":
    model = ResNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate, num_blocks=num_blocks).float()
    model.to(device)
    print("ResNet initialized")
    
elif model_choice == "ConvNet":
    model = ConvNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate).float()
    model.to(device)
    print("ConvNet initialized")

else:
    sys.exit("The chosen model isn't valid")

In [ ]:
model.load_state_dict(torch.load("../Models/{}".format(model_list[2]),map_location=device))
model.eval()

In [ ]:
for data, target,scaler in test_loader:
    data, scaler = data.to(device), scaler.to(device)
    with torch.no_grad():
        output = model(data, scaler)
    predicted = output.argmax(1).cpu()

    test_correct += (target == predicted).sum().item()

    c = (predicted == target).squeeze()
    for i in range(data.shape[0]):
        label = target[i]
        class_correct[label] += c[i].item()
        class_total[label] += 1
        
for i in range(len(classes)):
        print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
train_loader.sampler.num_samples = 10000

In [ ]:
start = time.time()
for data, target,scaler in train_loader:
    data, scaler = data.to(device), scaler.to(device)
    with torch.no_grad():
        output = model(data, scaler)
    predicted = output.argmax(1).cpu()
end = time.time()
diff = (end-start)
print(diff)